In [1]:
import pandas as pd
import numpy as np
import argparse

from pyproj import Transformer
from pathlib import Path

In [2]:
people_df = pd.read_csv("data/persons_home_depot_with_start_duration_draws.csv")
activities_df = pd.read_csv("data/activities_long_with_groups_selected_plus.csv")
charger_df = pd.read_csv("data/charger_location_attributes_dropped_final.csv")

/var/folders/p0/bcd_l4996lz24gnzf87fwjm80000gn/T/ipykernel_31467/3100424587.py:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  people_df = pd.read_csv("data/persons_home_depot_with_start_duration_draws.csv")


In [3]:
people_df.head()

,pid,x,y,hhincome,workstatus,gender,age_group,group,start_g1,dur_g1,...,start_g4,dur_g4,start_g5,dur_g5,start_g6,dur_g6,start_g7,dur_g7,start_g8,dur_g8
0,2002000535_3_E02001543_1_2002001259,453768.0,408587.0,low,False,male,65+,1,0.0,156.0,...,159.0,12.0,287.0,35.0,NaN,NaN,NaN,NaN,148.0,149.0
1,2002000535_3_E02001543_1_2002001260,453768.0,408587.0,low,False,female,65+,1,0.0,138.0,...,127.0,5.0,148.0,36.0,NaN,NaN,NaN,NaN,149.0,4.0
2,2002000535_5_E02001617_1_2002001259,437044.0,393175.0,low,False,male,65+,1,224.0,169.0,...,128.0,164.0,147.0,35.0,NaN,NaN,NaN,NaN,108.0,20.0
3,2002000535_5_E02001617_1_2002001260,437044.0,393175.0,low,False,female,65+,1,122.0,136.0,...,177.0,46.0,147.0,35.0,NaN,NaN,NaN,NaN,130.0,29.0
4,2002000536_7_E02001588_1_2002001261,440254.0,394149.0,low,False,female,65+,1,0.0,114.0,...,109.0,71.0,287.0,8.0,NaN,NaN,NaN,NaN,242.0,67.0


In [4]:
activities_df.head()

,pid,activity_idx,act_type,x,y,group,earliest_start,latest_start,max_duration,min_duration
0,2002000535_3_E02001543_1_2002001259,1,home,453768.000000,408587.000000,1,0,288,288,2
1,2002000535_3_E02001543_1_2002001259,2,other,453768.000000,408587.000000,8,108,216,120,2
2,2002000535_3_E02001543_1_2002001259,3,home,453768.000000,408587.000000,1,0,288,288,2
3,2002000535_3_E02001543_1_2002001260,1,home,453768.000000,408587.000000,1,0,288,288,2
4,2002000535_3_E02001543_1_2002001260,2,pt interaction,453920.826652,408260.138674,8,108,216,120,2


In [5]:
# unique_pairs = (
#     activities_df[["act_type", "group"]].drop_duplicates().sort_values("group")
# )
# unique_pairs

In [6]:
activities_df["pid"].unique()

array(['2002000535_3_E02001543_1_2002001259',
       '2002000535_3_E02001543_1_2002001260',
       '2002000535_5_E02001617_1_2002001259', ..., 'LGV_997', 'LGV_998',
       'LGV_999'], shape=(131262,), dtype=object)

In [7]:
pids_with_2_or_5 = activities_df[activities_df["group"].isin([2, 5])]["pid"].unique()
pids_with_4 = activities_df[activities_df["group"].isin([4])]["pid"].unique()
pids_with_8 = activities_df[activities_df["group"].isin([8])]["pid"].unique()

In [8]:
pids_with_both = set(pids_with_2_or_5).intersection(set(pids_with_4))
pids_with_all = set(pids_with_both).intersection(set(pids_with_8))

In [9]:
pids_with_all

{'2006008764_2_E02001564_1_2006020931',
 '2017001054_3_E02001562_1_2017002497',
 '2016007062_5_E02001553_1_2016017054',
 '2010000264_7_E02006844_1_2010000618',
 '2008007698_3_E02001525_1_2008018252',
 '2007000283_4_E02001666_1_2007000677',
 '2017004615_4_E02001576_1_2017011031',
 '2018006686_7_E02001678_1_2018015921',
 '2006008073_5_E02002478_1_2006019275',
 '2012004698_7_E02001562_1_2012011083',
 '2010007669_4_E02001599_1_2010018242',
 '2016000144_2_E02001642_1_2016000340',
 '2013006062_5_E02001525_1_2013014530',
 '2009007908_4_E02001543_1_2009018982',
 '2015002218_7_E02002283_1_2015005377',
 '2016005529_1_E02005817_1_2016013272',
 '2002002693_5_E02001543_1_2002006282',
 '2003001570_5_E02001576_1_2003003840',
 '2017001564_3_E02001643_1_2017003747',
 '2013001538_5_E02001643_1_2013003739',
 '2006001016_5_E02001576_1_2006002504',
 '2003003094_4_E02001543_1_2003007542',
 '2017000457_7_E02001652_1_2017001044',
 '2010006258_3_E02001593_1_2010014847',
 '2008002477_6_E02001538_1_2008005839',


In [10]:
potential_individuals = [
    "2016005585_5_E02001538_1_2016013411",
    "2008003221_6_E02001532_1_2008007614",
    "2006004854_4_E02001609_1_2006011654",
    "2002000535_5_E02001617_1_2002001259",
    "2002007185_4_E02001606_1_2002016833"
]

In [11]:
person_259_df = activities_df[activities_df["pid"] == "2002000535_5_E02001617_1_2002001259"].drop('pid', axis = 1)
person_259_df.to_csv("data/test/person_259_acts.csv", index=False)

In [12]:
person_1_acts = activities_df[
    activities_df["pid"] == "2006004854_4_E02001609_1_2006011654"
]

In [13]:
person_259 = people_df[people_df["pid"] == "2002000535_5_E02001617_1_2002001259"]

In [14]:
person_1 = people_df[people_df["pid"] == "2006004854_4_E02001609_1_2006011654"]

In [15]:
charger_df.head()

,index,lat,lon,device_id,port_count,port_id,address1,city,network_name,power_range_max,power_imputed
0,0,59.034786,-2.905301,1000335565,2,1000335565_1,"Shapinsay Development Trust, Balfour",Orkney,ChargePlace Scotland,22.0,True
1,1,59.034786,-2.905301,1000335565,2,1000335565_2,"Shapinsay Development Trust, Balfour",Orkney,ChargePlace Scotland,200.0,True
2,2,58.982254,-2.973739,1000339545,2,1000339545_1,Muddisdale Road,Kirkwall,"Charge Your Car,ChargePlace Scotland",50.0,True
3,3,58.982254,-2.973739,1000339545,2,1000339545_2,Muddisdale Road,Kirkwall,"Charge Your Car,ChargePlace Scotland",22.0,True
4,4,58.981014,-2.965392,1000339745,3,1000339745_1,"South Car Park, Great Western Road",Kirkwall,"Charge Your Car,ChargePlace Scotland",7.0,True


In [16]:
def get_charge_mode(charge_power):
    if charge_power <= 7.0:
        return 1
    elif charge_power > 7.0 and charge_power <= 22.0:
        return 2
    else:
        return 3

In [17]:
charger_df["charge_mode"] = charger_df["power_range_max"].apply(get_charge_mode)
charger_df.to_csv("data/chargers_with_mode.csv", index=False)
charger_df

,index,lat,lon,device_id,port_count,port_id,address1,city,network_name,power_range_max,power_imputed,charge_mode
0,0,59.034786,-2.905301,1000335565,2,1000335565_1,"Shapinsay Development Trust, Balfour",Orkney,ChargePlace Scotland,22.0,True,2
1,1,59.034786,-2.905301,1000335565,2,1000335565_2,"Shapinsay Development Trust, Balfour",Orkney,ChargePlace Scotland,200.0,True,3
2,2,58.982254,-2.973739,1000339545,2,1000339545_1,Muddisdale Road,Kirkwall,"Charge Your Car,ChargePlace Scotland",50.0,True,3
3,3,58.982254,-2.973739,1000339545,2,1000339545_2,Muddisdale Road,Kirkwall,"Charge Your Car,ChargePlace Scotland",22.0,True,2
4,4,58.981014,-2.965392,1000339745,3,1000339745_1,"South Car Park, Great Western Road",Kirkwall,"Charge Your Car,ChargePlace Scotland",7.0,True,1
...,...,...,...,...,...,...,...,...,...,...,...,...
32656,36138,51.510784,-0.186585,1010414445,1,1010414445_1,"Q-Park Queensway, London 4",London,GreenFlux Netherlands,7.4,False,2
32657,36139,51.510796,-0.186594,1010414435,1,1010414435_1,"Q-Park Queensway, London 3",London,GreenFlux Netherlands,7.4,False,2
32658,36140,51.510830,-0.186600,1010414485,1,1010414485_1,"Q-Park Queensway, London 2",London,GreenFlux Netherlands,7.4,False,2
32659,36141,51.510849,-0.186606,1010414495,1,1010414495_1,"Q-Park Queensway, London 1",London,GreenFlux Netherlands,7.4,False,2


In [16]:
person_1.to_csv("data/test/person_1.csv", index=False)

In [17]:
person_1_acts.to_csv("data/test/person_1_acts.csv", index=False)

In [18]:
def create_test_individual_csv(person, output_dir):
    """Create a simplified individual CSV for testing."""
    # Mapping from DP group numbers to activity type names
    group_to_type = {
        1: "home",
        2: "work",
        3: "business",
        4: "shop/visit",
        5: "education",
        6: "depot/medical",
        7: "delivery/visit",
        8: "other/escort",
    }

    individual_data = {
        "pid": person["pid"].values[0],
        "home_x": person["x"].values[0],
        "home_y": person["y"].values[0],
        "work_x": person["x"].values[0],  # Simplified: use home as work
        "work_y": person["y"].values[0],
        "age_group": person["age_group"].values[0],
        "workstatus": person["workstatus"].values[0],
    }

    # Add start times and durations for each group
    for group in range(1, 9):
        if group in group_to_type:
            activity_type = group_to_type[group]

            start_col = f"start_g{group}"
            dur_col = f"dur_g{group}"

            if start_col in person.columns:
                individual_data[f"{activity_type}_start"] = (
                    person[start_col].values[0]
                    if not pd.isna(person[start_col].values[0])
                    else 0
                )
            else:
                individual_data[f"{activity_type}_start"] = 0

            if dur_col in person.columns:
                individual_data[f"{activity_type}_dur"] = (
                    person[dur_col].values[0]
                    if not pd.isna(person[dur_col].values[0])
                    else 12
                )
            else:
                individual_data[f"{activity_type}_dur"] = 12

    individual_df = pd.DataFrame([individual_data])
    output_path = output_dir + "test_individual.csv"
    individual_df.to_csv(output_path, index=False)

    print(f"\nSaved individual data to {output_path}")

    return individual_df

In [86]:
create_test_individual_csv(person_1, "data/test/")


Saved individual data to data/test/test_individual.csv


,pid,home_x,home_y,work_x,work_y,age_group,workstatus,home_start,home_dur,work_start,...,shop/visit_start,shop/visit_dur,education_start,education_dur,depot/medical_start,depot/medical_dur,delivery/visit_start,delivery/visit_dur,other/escort_start,other/escort_dur
0,2006004854_4_E02001609_1_2006011654,454070.0,382249.0,454070.0,382249.0,21-40,True,227.0,39.0,168.0,...,141.0,55.0,104.0,84.0,0,12,0,12,220.0,82.0


In [20]:
charger_df.head()

,index,lat,lon,device_id,port_count,port_id,address1,city,network_name,power_range_max,power_imputed,charge_mode
0,0,59.034786,-2.905301,1000335565,2,1000335565_1,"Shapinsay Development Trust, Balfour",Orkney,ChargePlace Scotland,22.0,True,2
1,1,59.034786,-2.905301,1000335565,2,1000335565_2,"Shapinsay Development Trust, Balfour",Orkney,ChargePlace Scotland,200.0,True,3
2,2,58.982254,-2.973739,1000339545,2,1000339545_1,Muddisdale Road,Kirkwall,"Charge Your Car,ChargePlace Scotland",50.0,True,3
3,3,58.982254,-2.973739,1000339545,2,1000339545_2,Muddisdale Road,Kirkwall,"Charge Your Car,ChargePlace Scotland",22.0,True,2
4,4,58.981014,-2.965392,1000339745,3,1000339745_1,"South Car Park, Great Western Road",Kirkwall,"Charge Your Car,ChargePlace Scotland",7.0,True,1


In [21]:
activities_df.head()

,pid,activity_idx,act_type,x,y,group,earliest_start,latest_start,max_duration,min_duration
0,2002000535_3_E02001543_1_2002001259,1,home,453768.000000,408587.000000,1,0,288,288,2
1,2002000535_3_E02001543_1_2002001259,2,other,453768.000000,408587.000000,8,108,216,120,2
2,2002000535_3_E02001543_1_2002001259,3,home,453768.000000,408587.000000,1,0,288,288,2
3,2002000535_3_E02001543_1_2002001260,1,home,453768.000000,408587.000000,1,0,288,288,2
4,2002000535_3_E02001543_1_2002001260,2,pt interaction,453920.826652,408260.138674,8,108,216,120,2


In [22]:
person_1_acts

,pid,activity_idx,act_type,x,y,group,earliest_start,latest_start,max_duration,min_duration
160581,2006004854_4_E02001609_1_2006011654,1,home,454070.0,382249.0,1,0,288,288,2
160582,2006004854_4_E02001609_1_2006011654,2,other,452811.0,385797.0,8,108,216,120,2
160583,2006004854_4_E02001609_1_2006011654,3,visit,452551.0,385259.0,4,84,276,120,2
160584,2006004854_4_E02001609_1_2006011654,4,escort_shop,452211.0,383737.0,8,108,216,120,2
160585,2006004854_4_E02001609_1_2006011654,5,home,454070.0,382249.0,1,0,288,288,2
160586,2006004854_4_E02001609_1_2006011654,6,other,456492.0,382027.0,8,108,216,120,2
160587,2006004854_4_E02001609_1_2006011654,7,work,474270.0,381532.0,2,60,276,144,2
160588,2006004854_4_E02001609_1_2006011654,8,shop,467941.0,378919.0,4,84,276,120,2
160589,2006004854_4_E02001609_1_2006011654,9,home,454070.0,382249.0,1,0,288,288,2


In [23]:
lat = 59.034786
long = -2.905301
transformer_EN = Transformer.from_crs("EPSG:4326", "EPSG:27700")
transformer_latlong = Transformer.from_crs("EPSG:27700","EPSG:4326")

In [24]:
easting, northing = transformer_EN.transform(lat, long)

print(f"easting {easting}, northing {northing}")

easting 348140.4097676404, northing 1016769.9346357614


In [25]:
lat, long = transformer_latlong.transform(easting, northing)

In [26]:
print(f"lat {lat}, long {long}")

lat 59.034786, long -2.905301000000004


In [27]:
charger_df['x'], charger_df['y'] = transformer_EN.transform(charger_df['lat'], charger_df['lon'])

In [28]:
charger_df

,index,lat,lon,device_id,port_count,port_id,address1,city,network_name,power_range_max,power_imputed,charge_mode,x,y
0,0,59.034786,-2.905301,1000335565,2,1000335565_1,"Shapinsay Development Trust, Balfour",Orkney,ChargePlace Scotland,22.0,True,2,348140.404705,1.016770e+06
1,1,59.034786,-2.905301,1000335565,2,1000335565_2,"Shapinsay Development Trust, Balfour",Orkney,ChargePlace Scotland,200.0,True,3,348140.404705,1.016770e+06
2,2,58.982254,-2.973739,1000339545,2,1000339545_1,Muddisdale Road,Kirkwall,"Charge Your Car,ChargePlace Scotland",50.0,True,3,344127.360009,1.010976e+06
3,3,58.982254,-2.973739,1000339545,2,1000339545_2,Muddisdale Road,Kirkwall,"Charge Your Car,ChargePlace Scotland",22.0,True,2,344127.360009,1.010976e+06
4,4,58.981014,-2.965392,1000339745,3,1000339745_1,"South Car Park, Great Western Road",Kirkwall,"Charge Your Car,ChargePlace Scotland",7.0,True,1,344605.132031,1.010831e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32656,36138,51.510784,-0.186585,1010414445,1,1010414445_1,"Q-Park Queensway, London 4",London,GreenFlux Netherlands,7.4,False,2,525938.330356,1.806535e+05
32657,36139,51.510796,-0.186594,1010414435,1,1010414435_1,"Q-Park Queensway, London 3",London,GreenFlux Netherlands,7.4,False,2,525937.674312,1.806548e+05
32658,36140,51.510830,-0.186600,1010414485,1,1010414485_1,"Q-Park Queensway, London 2",London,GreenFlux Netherlands,7.4,False,2,525937.162889,1.806586e+05
32659,36141,51.510849,-0.186606,1010414495,1,1010414495_1,"Q-Park Queensway, London 1",London,GreenFlux Netherlands,7.4,False,2,525936.693633,1.806607e+05


In [30]:
for _, act in person_1_acts.iterrows():
    act_x, act_y = act['x'], act['y']
    charger_df["distance"] = np.sqrt((charger_df['x']-act_x)**2 + (charger_df['y']-act_y)**2)

In [33]:
charger_df = charger_df.drop('power_imputed', axis=1)

In [34]:
charger_df

,index,lat,lon,device_id,port_count,port_id,address1,city,network_name,power_range_max,charge_mode,x,y,distance
0,0,59.034786,-2.905301,1000335565,2,1000335565_1,"Shapinsay Development Trust, Balfour",Orkney,ChargePlace Scotland,22.0,2,348140.404705,1.016770e+06,643302.360348
1,1,59.034786,-2.905301,1000335565,2,1000335565_2,"Shapinsay Development Trust, Balfour",Orkney,ChargePlace Scotland,200.0,3,348140.404705,1.016770e+06,643302.360348
2,2,58.982254,-2.973739,1000339545,2,1000339545_1,Muddisdale Road,Kirkwall,"Charge Your Car,ChargePlace Scotland",50.0,3,344127.360009,1.010976e+06,638267.211368
3,3,58.982254,-2.973739,1000339545,2,1000339545_2,Muddisdale Road,Kirkwall,"Charge Your Car,ChargePlace Scotland",22.0,2,344127.360009,1.010976e+06,638267.211368
4,4,58.981014,-2.965392,1000339745,3,1000339745_1,"South Car Park, Great Western Road",Kirkwall,"Charge Your Car,ChargePlace Scotland",7.0,1,344605.132031,1.010831e+06,638042.229237
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32656,36138,51.510784,-0.186585,1010414445,1,1010414445_1,"Q-Park Queensway, London 4",London,GreenFlux Netherlands,7.4,2,525938.330356,1.806535e+05,214022.887610
32657,36139,51.510796,-0.186594,1010414435,1,1010414435_1,"Q-Park Queensway, London 3",London,GreenFlux Netherlands,7.4,2,525937.674312,1.806548e+05,214021.483607
32658,36140,51.510830,-0.186600,1010414485,1,1010414485_1,"Q-Park Queensway, London 2",London,GreenFlux Netherlands,7.4,2,525937.162889,1.806586e+05,214017.724664
32659,36141,51.510849,-0.186606,1010414495,1,1010414495_1,"Q-Park Queensway, London 1",London,GreenFlux Netherlands,7.4,2,525936.693633,1.806607e+05,214015.579332


In [62]:
nearby = charger_df[charger_df['distance']<=20000]

In [66]:
charger_df[charger_df['city']=="Sheffield"]

,index,lat,lon,device_id,port_count,port_id,address1,city,network_name,power_range_max,charge_mode,x,y,distance
6450,6450,53.525627,-1.627380,1001397615,2,1001397615_1,Penistone,Sheffield,Chargemaster (POLAR),50.00,3,424800.125354,403280.641272,36042.412466
6451,6451,53.525627,-1.627380,1001397615,2,1001397615_2,Penistone,Sheffield,Chargemaster (POLAR),22.00,2,424800.125354,403280.641272,36042.412466
7218,7218,53.461644,-1.491865,6712215,2,6712215_1,Acorn,Sheffield,RAW Charging,22.08,3,433834.554274,396217.786791,24588.620705
7219,7219,53.461644,-1.491865,6712215,2,6712215_2,Acorn,Sheffield,RAW Charging,22.08,3,433834.554274,396217.786791,24588.620705
7220,7220,53.461647,-1.491792,6712235,2,6712235_1,Acorn,Sheffield,RAW Charging,38.40,3,433839.396831,396218.136172,24584.834146
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8839,8839,53.315617,-1.281415,1000356985,2,1000356985_2,"M1 Motorway Southbound, Jct 30/31,Woodall",Sheffield,Ecotricity (Electric Highway),22.00,2,447970.582499,380092.039067,6469.572963
8866,8866,53.315159,-1.282853,1000356465,2,1000356465_1,"M1 Motorway Northbound, Jct 30/31,Woodall",Sheffield,Ecotricity (Electric Highway),3.00,1,447875.292542,380040.149956,6576.733156
8867,8867,53.315159,-1.282853,1000356465,2,1000356465_2,"M1 Motorway Northbound, Jct 30/31,Woodall",Sheffield,Ecotricity (Electric Highway),7.00,1,447875.292542,380040.149956,6576.733156
8868,8868,53.315159,-1.282853,1000358355,2,1000358355_1,"M1 Motorway Northbound, Jct 30/31,Woodall",Sheffield,Ecotricity (Electric Highway),200.00,3,447875.292542,380040.149956,6576.733156


In [61]:
200*5/60

16.666666666666668